In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
# Graph containing places
deep_reload(src)

filepath = 'Data/Places/tl_2023_06_place.shp'
node_attributes = {
    'name': 'lambda n: n["NAME"]',
    'class': 'lambda n: n["CLASSFP"]',
    'geoid': 'lambda n: n["GEOID"]',
    'x': 'lambda n: n["geometry"].centroid.x',
    'y': 'lambda n: n["geometry"].centroid.y',
    'type': 'lambda n: "place"',
}

graph_place = src.graph.graph_from_shapefile(
    filepath, node_attributes, contains_links = False)

mapping = {n: f'place_{n}' for n in graph_place.nodes}

graph_place = nx.relabel_nodes(graph_place, mapping)

In [3]:
# Graph containing DC chargers

with open('Data/AFDC/evse_stations.json', 'r') as file:
    evse = json.load(file)

In [4]:
stations_raw = []

for station in evse['fuel_stations']:

    checks = (
        station['state'] == 'CA',
        station['ev_dc_fast_num'] is not None,
        # station['ev_dc_fast_num'] > 0,
        station['access_code'] == 'public',
        station['status_code'] == 'E',
        
    )

    if all(checks):

        stations_raw.append(station)

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [5]:
df_stations = pd.DataFrame.from_dict(stations_raw)

In [6]:
node_attributes = {
    'x': 'lambda n: n["longitude"]',
    'y': 'lambda n: n["latitude"]',
    'n_dcfc': 'lambda n: n["ev_dc_fast_num"]',
    'n_acl2': 'lambda n: n["ev_level2_evse_num"]',
    'network': 'lambda n: n["ev_network"]',
    'name': 'lambda n: n["station_name"]',
    'address': 'lambda n: n["street_address"]',
    'city': 'lambda n: n["city"]',
    'state': 'lambda n: n["state"]',
    'zip': 'lambda n: n["zip"]',
    'access_code': 'lambda n: n["access_code"]',
    'status_code': 'lambda n: n["status_code"]',
    'type': 'lambda n: "station"',
}

nlg = src.graph.nlg_from_dataframe(df_stations, node_attributes)

graph_station = src.graph.graph_from_nlg(nlg)

mapping = {n: f'station_{n}' for n in graph_station.nodes}

graph_station = nx.relabel_nodes(graph_station, mapping)

In [7]:
graph = nx.union(graph_place, graph_station)

In [8]:
graph._node['place_0'], graph._node['station_0']

({'status': 0,
  'visited': 0,
  'name': 'San Fernando',
  'class': 'C1',
  'geoid': '0666140',
  'x': -118.436242283817,
  'y': 34.28865249013806,
  'type': 'place'},
 {'status': 0,
  'visited': 0,
  'x': -118.4898,
  'y': 34.010528,
  'n_dcfc': 2,
  'n_acl2': 12.0,
  'network': 'Non-Networked',
  'name': 'City of Santa Monica - Civic Parking Garage',
  'address': '333 Civic Center Dr',
  'city': 'Santa Monica',
  'state': 'CA',
  'zip': '90401',
  'access_code': 'public',
  'status_code': 'E',
  'type': 'station'})

In [9]:
atlas = src.graph.graph_from_json('atlas.json')

FileNotFoundError: [Errno 2] No such file or directory: 'atlas.json'

In [12]:
weights = {
    "length": 400e3,
    "time": 28800,
}

graph = src.adjacency.adjacency(atlas, graph, weights, compute_all = True)

 [------->] (3524/3524) 100.00%, 6.94 it/s, 508.08 s elapsed, 0.00 s remaining

In [13]:
filename = 'Outputs/entity_atlas.json'

src.graph.graph_to_json(graph, filename)